In [17]:
import pandas as pd
import langchain
from langchain_openai import ChatOpenAI
from langchain.chains import ConversationChain
from langchain.memory import ConversationBufferMemory, ConversationBufferWindowMemory, ConversationSummaryMemory, ConversationSummaryBufferMemory
from langchain.memory import ChatMessageHistory
from langchain.schema import message_to_dict
from langchain.callbacks import get_openai_callback
import threading
import json
import time
from collections import OrderedDict
from langchain_openai import OpenAI

from langchain.prompts.chat import (
            ChatPromptTemplate,
            SystemMessagePromptTemplate,
            AIMessagePromptTemplate,
            HumanMessagePromptTemplate,
            MessagesPlaceholder
        )
from langchain.chains import LLMChain
from langchain_core.messages import AIMessage, HumanMessage
import re
import os


In [3]:
client = OpenAI(
    api_key =  ##prem key
)

In [6]:
llm = ChatOpenAI(temperature=0,
             openai_api_key = ##prem key,
             model = "gpt-3.5-turbo-1106")
conversation_buffer = ConversationChain(llm=llm,
                                 memory=ConversationBufferMemory())
conversation_buffer.memory.clear()
conversation_buffer.memory
conversation_buffer.memory.chat_memory.messages

[]

In [77]:
with open('search_data_simple_GDPR.txt', 'r') as file:
    codes_questions = file.readlines()

codes_questions
questions = [item.strip('\n\t') for item in codes_questions]
questions
questions_for_answering = {}
 
for quest in range(0, len(questions), 2):
    code = questions[quest].strip(':')
    question = questions[quest+1]
    questions_for_answering[code] = question


In [10]:
with open('abstract_policies.json', 'r', encoding='utf=8') as file:
    abstracts_json = json.load(file)

In [106]:
def define_messages_prompt_template(policy, questions_for_answering):
    system_message_prompt = SystemMessagePromptTemplate.from_template("This is a comprehensive conversation in which questions about a specific policy are answered")
    example_gpt_answer = { "GDPR1": {
                    "Answer": "Yes",
                    "Extract": "Companies which do not have their seat in the EEA should appoint a representative within the EU."
                },
                "GDPR2": {
                    "Answer": "Yes",
                    "Extract": "We collect this information to better understand how people use our websites and to see how our websites perform on different devices."
                },
                "GDPR3": {
                    "Answer": "Yes",
                    "Extract": "We collect this information to better understand how people use our websites and to see how our websites perform on different devices."
                },
                "GDPR4": {
                    "Answer": "No",
                    "Extract": ""
                },
                "GDPR5": {
                    "Answer": "Yes",
                    "Extract": "The information we collect from all visitors to our website may include your device, operating system, browser type, referring site, and the date and time of each visit."
                },
                "GDPR6": {
                    "Answer": "Yes",
                    "Extract": "We keep personal information only for as long as required for the purposes for which it is supplied."
                },
                "GDPR7": {
                    "Answer": "Yes",
                    "Extract": "We do not allow any outside parties to access the database."
                },
                "GDPR8": {
                    "Answer": "Yes",
                    "Extract": "This Privacy Policy was last updated on 15 March 2023."
                },
                "GDPR9": {
                    "Answer": "Yes",
                    "Extract": "We use common internet tools such as cookies, web server logs, and analytics services to gather the data."
                },
                "GDPR10": {
                    "Answer": "Yes",
                    "Extract": "If you set your browser to block all tracking this is fine, everything will work."
                },
                "GDPR11": {
                    "Answer": "Yes",
                    "Extract": "We have policies and other controls in place to provide appropriate protection of customers\u2019 personal data, based on a careful assessment of risk."
                },
                "GDPR12": {
                    "Answer": "Yes",
                    "Extract": "We collect this information to better understand how people use our websites and to see how our websites perform on different devices."
                },
                "GDPR13": {
                    "Answer": "Yes",
                    "Extract": "We collect this information to better understand how people use our websites and to see how our websites perform on different devices."
                },
                "GDPR14": {
                    "Answer": "Yes",
                    "Extract": "You, as an individual or in your capacity within an institution, have rights around the use of your data, including the ability to opt out of promotional emails, cookies, and collection of data by certain analytics providers."
                },
                "GDPR15": {
                    "Answer": "Yes",
                    "Extract": "We keep personal information only for as long as required for the purposes for which it is supplied."
                },
                "GDPR16": {
                    "Answer": "Yes",
                    "Extract": "We collect this information to better understand how people use our websites and to see how our websites perform on different devices."
                },
                "GDPR17": {
                    "Answer": "Yes",
                    "Extract": "We use appropriate security based on the type and sensitivity of data being stored."
                },
                "GDPR18": {
                    "Answer": "Yes",
                    "Extract": "We collect certain data from you directly, such as information you enter yourself, which includes your name, your role (e.g. teacher, librarian, student, etc.), your country, your email address and your institution."
                },
                "GDPR19": {
                    "Answer": "Yes",
                    "Extract": "We collect certain data from you directly, such as information you enter yourself, which includes your name, your role (e.g. teacher, librarian, student, etc.), your country, your email address and your institution."
                },
                "GDPR20": {
                    "Answer": "Yes",
                    "Extract": "We keep personal information only for as long as required for the purposes for which it is supplied."
                },
                "GDPR21": {
                    "Answer": "Yes",
                    "Extract": "If you have any questions, please contact our IT Manager, Martin Ng, who is in overall charge of privacy and data protection: martin.ng@clarityenglish.com."
                },
                "GDPR22": {
                    "Answer": "Yes",
                    "Extract": "You, as an individual or in your capacity within an institution, have rights around the use of your data, including the ability to opt out of promotional emails, cookies, and collection of data by certain analytics providers."
                },
                "GDPR23": {
                    "Answer": "No",
                    "Extract": ""
                },
                "GDPR24": {
                    "Answer": "Yes",
                    "Extract": "You, as an individual or in your capacity within an institution, have rights around the use of your data, including the ability to opt out of promotional emails, cookies, and collection of data by certain analytics providers."
                },
                "GDPR25": {
                    "Answer": "Yes",
                    "Extract": "You, as an individual or in your capacity within an institution, have rights around the use of your data, including the ability to opt out of promotional emails, cookies, and collection of data by certain analytics providers."
                },
                "GDPR26": {
                    "Answer": "No",
                    "Extract": ""
                },
                "GDPR27": {
                    "Answer": "No",
                    "Extract": ""
                },
                "GDPR28": {
                    "Answer": "No",
                    "Extract": ""
                },
                "GDPR29": {
                    "Answer": "Yes",
                    "Extract": "You, as an individual or in your capacity within an institution, have rights around the use of your data, including the ability to opt out of promotional emails, cookies, and collection of data by certain analytics providers."
                },
                "GDPR30": {
                    "Answer": "Yes",
                    "Extract": "You, as an individual or in your capacity within an institution, have rights around the use of your data, including the ability to opt out of promotional emails, cookies, and collection of data by certain analytics providers."
                },
                "GDPR31": {
                    "Answer": "Yes",
                    "Extract": "You, as an individual or in your capacity within an institution, have rights around the use of your data, including the ability to opt out of promotional emails, cookies, and collection of data by certain analytics providers."
                },
                "GDPR32": {
                    "Answer": "Yes",
                    "Extract": "ClarityEnglish confirms that: A privacy policy is in place for the protection of personal data which has been approved by our own management and communicated to all employees and other relevant people (such as our worldwide distributors)."
                },
                "GDPR33": {
                    "Answer": "Yes",
                    "Extract": "ClarityEnglish Privacy Policy. This Privacy Policy was last updated on 15 March 2023."
                },
                "GDPR34": {
                    "Answer": "Yes",
                    "Extract": "We collect certain data from you directly, such as information you enter yourself, which includes your name, your role (e.g. teacher, librarian, student, etc.), your country, your email address and your institution."
                },
                "GDPR35": {
                    "Answer": "Yes",
                    "Extract": "You can use your browser to block all cookies and tracking \u2014 this will not stop our website or programs from working."
                },
                "GDPR36": {
                    "Answer": "No",
                    "Extract": ""
                },
                "GDPR37": {
                    "Answer": "Yes",
                    "Extract": "We do not allow any outside parties to access the database."
                },
                "GDPR38": {
                    "Answer": "No",
                    "Extract": ""
                },
                "GDPR39": {
                    "Answer": "Yes",
                    "Extract": "If you set your browser to block all tracking this is fine, everything will work."
                },
                "GDPR40": {
                    "Answer": "Yes",
                    "Extract": "We will amend this Privacy Policy from time to time."
                },
                "GDPR41": {
                    "Answer": "Yes",
                    "Extract": "We collect certain data from you directly, such as information you enter yourself."
                },
                "GDPR42": {
                    "Answer": "Yes",
                    "Extract": "We have updated this policy to make it shorter and easier to read, and to comply with the General Data Protection Regulation (GDPR)."
                },
                "GDPR43": {
                    "Answer": "Yes",
                    "Extract": "Where information on action taken on data subject rights (Article 15-22) cannot be provided within one month of receipt, does the privacy policy inform data subjects of the required extension within one month, include reasons for the delay, and extend delivery no more than two months."
                },
                "GDPR44": {
                    "Answer": "Yes",
                    "Extract": "We use appropriate security based on the type and sensitivity of data being stored."
                },
                "GDPR45": {
                    "Answer": "Yes",
                    "Extract": "We do not allow any outside parties to access the database."
                }
                 }
    
    example_gpt_answer_specific_questions = {}
    for question in questions_for_answering:
        if question in example_gpt_answer:
            example_gpt_answer_specific_questions[question] = example_gpt_answer[question]
    example_gpt_answer_string = json.dumps(example_gpt_answer_specific_questions, indent=4)
    
    example_abstract_prompt = """ Privacy policy | ClarityEnglish .Programs.Events.Blog.Support.Contact us.Sign in.Price enquiry.ClarityEnglish Privacy Policy.This Privacy Policy was last updated on 15 March 2023.We have updated this policy to make it shorter and easier to read, and to comply with the General Data Protection Regulation (GDPR).Unless we link to a different policy or state otherwise, this Privacy Policy applies when you visit or use the ClarityEnglish website, applications, APIs or other related services. By using our services, you consent to such use of your information and Personal Data and agree to the terms of this Privacy Policy.The data protection principle applies to all data entered by visitors to any ClarityEnglish website, by an institution about their students, and by students themselves. It also applies to progress records generated by the ClarityEnglish programs and stored in the database.1. Privacy when browsing ClarityEnglish.com.What data do we collect.If you are browsing our website and not signing in, we collect the same basic information that most websites collect. We use common internet tools such as cookies, web server logs, and analytics services to gather the data. If you set your browser to block all tracking this is fine, everything will work. We have no third-party advertising whatsoever on the website.The information we collect from all visitors to our website may include your device, operating system, browser type, referring site, and the date and time of each visit. We may also collect potentially personally-identifying information like Internet Protocol (IP) addresses and your country of origin.Why do we collect this data.We collect this information to better understand how people use our websites and to see how our websites perform on different devices.What do we use your data for.All data is available to a limited number of ClarityEnglish staff for support, research, and development work. We do not allow any outside parties to access the database.Any personal information provided by you is used only to help us provide a better service to you and for us to identify your needs and requirements.By using Google Analytics we know that your tracking data is also collected by them and used within their guidelines.Information collected from the enquiry form.We collect certain data from you directly, such as information you enter yourself, which includes your name, your role (e.g. teacher, librarian, student, etc.), your country, your email address and your institution.We collect this information to better understand how to respond to your enquiry, and to provide information that we hope will be of interest to you. We use your email address to communicate further with you. If we have a distributor in your country or region, we may pass your details on to them so that they can contact you.All data is available to a limited number of ClarityEnglish staff for support, research, and development work. In the case of institution enquiries, if we have a distributor in your territory, we will forward the form you have filled in to them so that they can answer your enquiry. Any personal information provided by you is used only to help us provide a better service to you and for us to identify your needs and requirements. No third party sees this information.2. Privacy as a subscriber to ClarityEnglish.com.When you or an institution that you are part of subscribe to a ClarityEnglish program you have to accept our terms and conditions of use, which are detailed here. The data that we collect from and about you is subject to the following privacy policy. By using our services, you consent to such use of your information and Personal Data and agree to the terms of this Privacy Policy.The data protection principle applies to all data entered by visitors to any ClarityEnglish website, by an institution about their students, and by students themselves. It also applies to progress records generated by the ClarityEnglish programs and stored in a database.What data do we collect.If you (or your institution's administrators) have created an account, we require some basic information at the time of account creation. This includes either your name, your ID, or your email address, and your password.Why do we collect this data.We collect this information to create your account and provide services to you. We use your information to identify you in our programs so that we can provide you with information about your scores and activities in exercises in the ClarityEnglish programs. We may use your email address to communicate further with you. In all cases, we will inform you beforehand of the emails you should expect to receive. For example, if you are an administrator, you should expect to receive emails linking to usage charts. If you are a test taker, we might email you a certificate when the test is complete.What do we use your data for.In addition to the purposes described above, data is available to a limited number of ClarityEnglish staff for support, research, and development work. We do not allow any outside party to access this information.Who we share your data with.We do not share, sell, rent, or trade users' personal information with third parties for their commercial purposes, except as specified within this document. Details about users that are entered into ClarityEnglish programs are available to administrators and others with appropriate, licensed access. For example, teachers can access all details about their students using Admin Panel; a student can only see their own details.Security.We use appropriate security based on the type and sensitivity of data being stored. As with any internet-enabled system, there is always a risk of unauthorised access, so it is important to protect passwords and to contact us if you suspect any unauthorised access to your account.A teacher's access is restricted to data on students in their own group only as designated by the administrator. An account administrator can see all details for that account.When a user is deleted from the system, all records relating to that user are deleted or anonymised.How long do we keep your data for.We keep personal information only for as long as required for the purposes for which it is supplied. Where your information is no longer required, we will always make sure it is disposed of or anonymised in a safe and secure manner.For institutional accounts, data is kept for access by the administrator and other licensed parties within the institution for as long as the licence is valid and the user account is active. Where the user account becomes inactive, data will be deleted or anonymised a maximum of one year after the last login. An administrator can delete data at any time, which will cause it to be permanently and safely destroyed.Where an institutional licence lapses or terminates, data is kept for up to a year. This is so that it can be restored to the system should the institution choose to renew its subscription. After a maximum of a year the data will be deleted or anonymised.Where a member of a public library creates a personal account, data will be held for up to a year after the last login. This is so that library users can return to their accounts after a period away and can retrieve past performance data. After a year of inactivity, it will be deleted or anonymised.For licences through the British Council, which are not renewable, we remove personal information within one month of a subscription terminating. For licences purchased by individuals, which are renewable, we keep data active for one year after the subscription finishes.3. Personal rights and data removal or correction.You, as an individual or in your capacity within an institution, have rights around the use of your data, including the ability to opt out of promotional emails, cookies, and collection of data by certain analytics providers. You can update or terminate your account by contacting us with requests about your personal data.If you have a personal online subscription account, we can remove or change any of your details at any time. Please send an email to us and we will confirm your identity before proceeding with any changes. If you have a licence within an institutional account, please contact your institution to remove or change your details.If you have a British Council Road to IELTS licence, we can remove your account if you request it, but we cannot change details as these are usually provided to us directly from a British Council system.You can use your browser to block all cookies and tracking \u2014 this will not stop our website or programs from working. At the time of writing, Firefox is very good at doing this for you.4. ClarityEnglish compliance with the GDPR.The General Data Protection Regulation (GDPR) is a Regulation of the European Union and, from 25 May 2018, it applies to all organisations that collect and process the personal data of EU citizens.ClarityEnglish confirms that:.A privacy policy is in place for the protection of personal data which has been approved by our own management and communicated to all employees and other relevant people (such as our worldwide distributors).All ClarityEnglish employees have received awareness training regarding data protection and the GDPR.Everyone within ClarityEnglish understands their roles in the protection of personal data, and will receive training where needed.We are keeping records of processing as required by the GDPR.We do not use sub-processors.All of our employees are subject to confidentiality obligations with respect to personal data.When ClarityEnglish transfers any customers\u2019 personal data internationally, we will ensure that the transfer is legal under the GDPR.We have policies and other controls in place to provide appropriate protection of customers\u2019 personal data, based on a careful assessment of risk.5. Changes to the Privacy Policy.We will amend this Privacy Policy from time to time. Use of information we collect is subject to the Privacy Policy in effect at the time such information is used.6. Contacting ClarityEnglish.If you have any questions, please contact our IT Manager, Martin Ng, who is in overall charge of privacy and data protection: martin.ng@clarityenglish.com.Programs.Placement testing.General English.IELTS preparation.About.About us.Contact us.Blog.Events.Updates.Useful links.Price enquiry.Support.Terms and conditions.Privacy policy.\u00a9 1992\u20142024 ClarityEnglish """
    context_history = """For the following policy: """ + f"{example_abstract_prompt}"+ """ answer the following questions with a "Yes"/"No" answer and if the answer is "Yes" then provide an extract from the policy that is 
    LONGER than 200 characters and best fits the answer, otherwise return an empty string, nothing else that differs from this. Make sure that your response is only in a JSON format like this and DO NOT PROVIDE ANY ADDITIONAL TEXT: `{{"QuestionID": {{"Answer": "Your answer", "Extract": "Extract from the policy"}}}}`, where " Your Answer" represents your answer to the question, "Extract from the policy" is the best fit extract and 'QuestionID' is the id of the question that I provide you with.
    Below is the list of questions, in the following format `{{'QuestionID':'QuestionText'}}`: {questions_for_answering}""" 
    context = """For the following policy: {policy} answer the following questions with a "Yes"/"No" answer and if the answer is "Yes" then provide an extract from the policy that is 
    LONGER than 200 characters and best fits the answer, otherwise return an empty string, nothing else that differs from this. Make sure that your response is only in a JSON format like this and DO NOT PROVIDE ANY ADDITIONAL TEXT: `{{"QuestionID": {{"Answer": "Your answer", "Extract": "Extract from the policy"}}}}`, where " Your Answer" represents your answer to the question, "Extract from the policy" is the best fit extract and 'QuestionID' is the id of the question that I provide you with.
    Below is the list of questions, in the following format `{{'QuestionID':'QuestionText'}}`: {questions_for_answering}"""
    human_history_prompt = HumanMessagePromptTemplate.from_template(context_history)
    example_gpt_answer_string = example_gpt_answer_string.replace('{', '{{')
    example_gpt_answer_string = example_gpt_answer_string.replace('}', '}}')
    ai_response_prompt = AIMessagePromptTemplate.from_template(example_gpt_answer_string)
    human_prompt = HumanMessagePromptTemplate.from_template(context)

    messages = [
        system_message_prompt, human_history_prompt, ai_response_prompt, human_prompt
    ]

    chat_template = ChatPromptTemplate.from_messages(messages)
    template = chat_template.format_messages(policy=policy, questions_for_answering=questions_for_answering)
    return template

In [137]:
questions_for_answering = dict(list(questions_for_answering.items()))
test_policies = {}

for key, value in list(abstracts_json.items()):
    test_policies[key] = value

In [109]:
def add_matching_braces(string):
    stack = []
    unmatched_indices = []

    for i, char in enumerate(string):
        if char == '{':
            stack.append(i)
        elif char == '}':
            if not stack:
                unmatched_indices.append(i)
            else:
                stack.pop()

    for index in reversed(unmatched_indices):
        string = string[:index] + '}' + string[index:]

    while stack:
        string += '}'
        stack.pop()

    return string

In [161]:
def save_response_trial(policy_id, response, file_path):
    response_content = response if len(response) > 0 else "No response from the API"
    response_content = add_matching_braces(response_content)
    response_content = response_content.replace("\\'", "'")
    response_data = {"PolicyID": policy_id, "Response": json.loads(response_content)}

    responses = []

    if os.path.exists(file_path):
        with open(file_path, "r") as file:
            data = json.load(file)
            responses = data.get("responses", [])
        for r in responses:
            if r["PolicyID"] == policy_id:
                r["Response"].update(response_data["Response"])
                break
        else:
            responses.append(response_data)
        
    else:
        responses.append(response_data)

    with open(file_path, "w") as file:
        json.dump({"responses": responses}, file, indent=4)

In [25]:
def get_response_by_policy_id(data, policy_id):
    if len(data) > 0:
        for response in data["responses"]:
            if response["PolicyID"] == policy_id:
                return response["Response"]
    return None
def get_non_cached_questions(cached_questions, original_questions):
    missing = {}
    for key, value in original_questions.items():
        if cached_questions:
            if key not in cached_questions.keys():
                missing[key] = value
            else:
                prev_response = cached_questions[key]
                if prev_response['Answer'] == 'Yes' and len(prev_response['Extract']) < 200:
                    missing[key] = value
        else:
            return original_questions

    return missing

In [ ]:
prompt_responses = {}
file_path = '' #save responses to a file
answered_policies_ids = []
with open(file_path, "r") as file:
    response_data = json.load(file)
if len(response_data) > 0:
    for policy in response_data['responses']:
        answered_policies_ids.append(policy['PolicyID'])

for policyid, policy in test_policies.items():
    cached_answers = get_response_by_policy_id(response_data, policyid)
    missing_questions = get_non_cached_questions(cached_answers, questions_for_answering)

    if len(missing_questions) > 0:
        prompt_context = define_messages_prompt_template(policy, questions_for_answering=missing_questions)
        conversation_buffer.memory.clear()
        
        response = conversation_buffer.predict(input=prompt_context)
        save_response_trial(policyid, response, file_path)
        prompt_responses[policyid] = response